In [6]:
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox

""
#-------------------Space invader-------------------------#
#La classe qui permet de cree la frame (fenêtre) 
class SpaceInvaders(object):
    '''
    Main Game class
    '''
    def __init__(self): 
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both")
        self.game = Game(self.frame)
    def play(self):
        self.game.start_animation()
        self.root.mainloop()
        
#-------------------Game-------------------------#  
#qui permet de gerer le jeu(les animations, mouvement avec la methode de classe keypress)
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.canvas_height =800
        self.canvas_width = 1000
        self.canvas = tk.Canvas(self.frame, width=self.canvas_width, height=self.canvas_height,bg="black")
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
        self.canvas.grid()
        self.frame.winfo_toplevel().bind("<Key>", self.keypress)
   
    def keypress(self, event):#-------------------Gestion des touches -------------------------
        x1,y1,x2,y2=self.canvas.coords(self.defender.id)#--coordonne defender en appelant la fonction canvas.coords                                     
        if event.keysym == 'Left' and x1>0: 
            self.defender.move_in(self.canvas,-self.defender.move_delta)     #appel a la methode move_in de la classe defender
        elif event.keysym == 'Right' and x2<self.canvas_width:
            self.defender.move_in(self.canvas,self.defender.move_delta)
        elif event.keysym == 'space':
            self.defender.fire(self.canvas)
    def move_bullets(self):
        self.defender.move_bullets(self.canvas)
    def move_aliens_fleet(self):
        self.fleet.move_in(self.canvas)
    def manage_touched_aliens(self):
        self.fleet.manage_touched_aliens_by(self.canvas,self.defender)

    def animation(self):# Animation: bullet,flotte d'alien, et appel a la methode (gestion l'action de toucher un alien)
        self.move_bullets()
        self.move_aliens_fleet()
        self.manage_touched_aliens()
        self.frame.after(30, self.animation)
    def start_animation(self):#-------------------Debut des animation-------------------------
        self.animation()
        
        
        
        
class Defender(object):
    def __init__(self): 
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.speed = 8
        self.max_fired_bullets = 8
        self.fired_bullets = []
    def now_shoot(self):#acces au tableau de bullet 
        return self.fired_bullets
    def get_id():
        return self.id
    def get_coord(self,canvas):#-------------------coordonnee defender-------------------------
        return canvas.coords(self.id)
    def bullet_fire(self):
        return self.fired_bullets
    def install_in(self, canvas): #cree le defender
        self.canvas=canvas
        self.canvas_width=int(canvas.cget("width"))
        self.canvas_height=int(canvas.cget("height"))
        x,y=self.canvas_width//2,self.canvas_height-self.height
        sw=self.width
    
        self.id =self.canvas.create_rectangle(x, y, x+sw, y+sw, outline='black',fill='white')
    def move_in(self,canvas, dx):#-------------------move defender-------------------------
        self.canvas=canvas
        self.canvas.move(self.id,dx,0)
    def fire (self,canvas):#-------------------Gestion stock bullet-------------------------
        if(len(self.fired_bullets)<=self.max_fired_bullets):
            bullet=Bullet(self)
            bullet.install_in(canvas)
            self.fired_bullets.append(bullet)

    def move_bullets(self,canvas):# bouge bullet
        for i in range(0,len(self.fired_bullets)):
            x1,y1,x2,y2 = canvas.bbox(self.fired_bullets[i].id) #coord des balle
            if y1<0: #si le bullet dehors du cadre du jeu
                canvas.delete(self.fired_bullets[i].id) #suprression du dessin du bullet
                del self.fired_bullets[i] #delete de sa valeur du liste
                break
            else:
                self.fired_bullets[i].move_in(canvas)   #deplacement du bullet

        
        
        
class Bullet(object):
    def __init__(self, shooter):
        self.radius = 5
        self.color = "red"
        self.speed = 8
        self.id = None
        self.shooter = shooter
    def get_bullet(self):# acces au bullet
        return self.id

    def install_in(self, canvas):# cree un bullet
        self.canvas=canvas
        r = self.radius
        x = self.shooter.get_coord(canvas)[0] + self.shooter.width / 2
        y = self.shooter.get_coord(canvas)[1] - (self.shooter.get_coord(canvas)[1] - self.shooter.get_coord(canvas)[3])/2
        self.id=self.canvas.create_oval(x, y, x+r, y+r, outline='black',fill=self.color)

    def move_in(self, canvas):#-------------------deplacement bullet selon l'axe y-------------------------
        coord = canvas.coords(self.id)
        if coord[1] >= 0:
            canvas.move(self.id, 0, -self.speed)
        else:
            canvas.delete(self.id)
            #self.shooter.lost(self)


    
    
        #self.fleet.install_in(self.canvas)
    #def animation(self):
    #def start_animation(self):
    
    #def move_aliens_fleet(self):

class Alien(object):
    alive_image = None
    dead_image = None

    @classmethod
    def get_alive_image(cls):# acces image alien

        if cls.alive_image == None:
            cls.alive_image = tk.PhotoImage(file='alien.gif')
        return cls.alive_image
    
    @classmethod
    def get_dead_image(cls):# acces image alien mort soit explosion

        if cls.dead_image == None:
            cls.dead_image = tk.PhotoImage(file='explosion.gif')
        return cls.dead_image

    def __init__(self):
        self.id = None
        self.alive = True

    def is_alive(self):# acces au statu alien vivant
        return self.alive

    def touched_by(self, canvas, projectile):# gerrer l'action toucher l'alien a l'aide des coordonnes matricielle fourni par la ligne bbox
        self.alive = False
        bbox = canvas.coords(self.get_alien_id())
        canvas.delete(self.get_alien_id())
        self.install_dead_in(canvas, bbox[0], bbox[1])

    def get_alien_id(self):# acces alien mort
        return self.id
    
    def install_in(self, canvas, x, y, image, tag):# cree alien
        self.id = canvas.create_image(x, y, image=image, tags=tag)
        
    def install_alive_in(self, canvas, x, y):# cree alien vivant
        self.install_in(canvas, x, y, Alien.get_alive_image(), "alive_alien")

    def install_dead_in(self, canvas, x, y):# cree alien vivant
        self.install_in(canvas, x, y, Alien.get_dead_image(), "dead_alien")
        canvas.after(300, self.clean, canvas, self.get_alien_id())

    def move_in(self, canvas, dx, dy):
        if self.is_alive():
            canvas.move(self.get_alien_id(), dx, dy)

    def clean(self, canvas, canvas_id):
        canvas.delete(canvas_id)


class Fleet(object):
    def __init__(self):
        ''' Aliens '''
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 20
        self.alien_x_delta = 5
        self.alien_y_delta = 15
        fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * fleet_size

    def get_width(self):
        return ((Alien.get_alive_image().width() + self.aliens_inner_gap) * self.aliens_columns * 1.5)

    def install_in(self, canvas):
        x = 50
        y = self.aliens_inner_gap

        for i in range(len(self.aliens_fleet)):
            if (i % self.aliens_columns == 0):
                y = y + Alien.get_alive_image().height() + self.aliens_inner_gap
                x = 50
            alien = Alien()
            alien.install_alive_in(canvas, x, y)
            self.aliens_fleet[i] = alien
            x = x + Alien.get_alive_image().width() + self.aliens_inner_gap

    def move_in(self, canvas):
        bbox = canvas.bbox("alive_alien")
        if bbox == None: return
        cwidth = int(canvas.cget("width"))

        y_delta = 0
        if (self.alien_x_delta > 0):
            if (bbox[2] >= cwidth):
                self.alien_x_delta = -self.alien_x_delta 
                y_delta = self.alien_y_delta
        else:
            if (bbox[0] < 0):
                self.alien_x_delta = -self.alien_x_delta 
                y_delta = self.alien_y_delta

        for alien in self.aliens_fleet:
            alien.move_in(canvas, self.alien_x_delta, y_delta)

    def manage_touched_aliens_by(self, canvas, defender):
        bullets = defender.now_shoot()
        for bull in list(bullets):
            x1, y1, x2, y2 = canvas.coords(bull.get_bullet())
            touched = canvas.find_overlapping(x1, y1, x2, y2)
            for alien in self.aliens_fleet:
                if alien.is_alive() and alien.get_alien_id() in touched:
                    alien.touched_by(canvas, defender)
                    bullets.remove(bull)
                    canvas.delete(bull.get_bullet())
        




    
    
    
        


    #def fire(self, canvas):
    


x=SpaceInvaders()
x.play()

TclError: couldn't open "alien.gif": no such file or directory